## Machine Learning Project about the Adaboost Machine Learning Algorithm. 


#### Why have i chosen this part of the project?  Well i wanted to learn something new by implimenting an algorithm that we did not impliment in the class sessions hence that is why i chose Adaboost. 

### So why was Adaboost discovered?
The reason to why it was discovered was to deal with the issue of bais in weak classif